In [1]:
import numpy as np

 ### Labels

In [2]:
import os
def get_file_length(file_path):
    path, dirs, files = next(os.walk(file_path))
    count = len(files)
    #print(count)
    return count


In [3]:
def get_labels(c1,c2,c3,c4,c5):
    labels = np.zeros((c1,1))
    labels = np.concatenate((labels,np.ones((c2,1))))
    labels = np.concatenate((labels,2*np.ones((c3,1))))
    labels = np.concatenate((labels,3*np.ones((c4,1))))
    labels = np.concatenate((labels,4*np.ones((c5,1))))
    #print(labels.shape)
    return labels


In [4]:
# File Length 
belly_pain_count_test = get_file_length(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\test\belly_pain')
burping_count_test = get_file_length(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\test\burping')
discomfort_count_test = get_file_length(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\test\discomfort')
hungry_count_test = get_file_length(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\test\hungry')
tired_count_test = get_file_length(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\test\tired')
belly_pain_count_train = get_file_length(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\train\belly_pain')
burping_count_train = get_file_length(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\train\burping')
discomfort_count_train = get_file_length(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\train\discomfort')
hungry_count_train = get_file_length(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\train\hungry')
tired_count_train = get_file_length(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\train\tired')
belly_pain_count_valid = get_file_length(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\val\belly_pain')
burping_count_valid = get_file_length(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\val\burping')
discomfort_count_valid = get_file_length(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\val\discomfort')
hungry_count_valid = get_file_length(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\val\hungry')
tired_count_valid = get_file_length(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\val\tired')

In [5]:
labels_test = get_labels(belly_pain_count_test , burping_count_test , discomfort_count_test , hungry_count_test , tired_count_test)
labels_train = get_labels(belly_pain_count_train , burping_count_train , discomfort_count_train , hungry_count_train , tired_count_train)
labels_val = get_labels(belly_pain_count_valid , burping_count_valid , discomfort_count_valid , hungry_count_valid , tired_count_valid)

In [6]:
print(labels_test.shape)

(148, 1)


In [7]:
labels_train = np.append(labels_train,labels_val)

print(labels_train.shape)

(585,)


In [8]:
labels_train = np.reshape(labels_train,(labels_train.shape[0],1))
print(labels_train.shape)

(585, 1)


### 68 Features

In [9]:
#from python_speech_features import mfcc
#from python_speech_features import logfeatures
import scipy.io.wavfile as wav
import numpy
import os
import csv
import numpy as np


def get_features_array(dir):
    features = np.zeros((68,199,1))
    for folder in os.listdir(dir):
        folder_path = os.path.join(dir,folder)
        for filename in os.listdir(folder_path):
            if filename.endswith('.csv'):
                file_path = os.path.join(folder_path,filename)
                temp = np.genfromtxt(file_path, delimiter=",", skip_header = False)
                temp  = np.reshape(temp,(temp.shape[0],temp.shape[1],1))
                #print(temp.shape)
                features = np.append(features,temp,axis = 2)
                #print(mfcc.shape)
    features = np.reshape(features,(features.shape[2],features.shape[0],features.shape[1]))
    features = np.delete(features,0,axis = 0)
    return(features)

features_test = get_features_array(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\test')
print(features_test.shape)
features_train = get_features_array(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\train')
print(features_train.shape)
features_val = get_features_array(r'E:\Baby Cry\Baby Cry\donateacry-corpus-master\68features\val')
print(features_val.shape)    
        

(148, 68, 199)
(439, 68, 199)
(146, 68, 199)


### Classify 

In [10]:
features_train = np.append(features_train,features_val,axis = 0)
print(features_train.shape)

(585, 68, 199)


In [11]:
flatten_train = features_train.reshape(features_train.shape[0],-1)
flatten_test = features_test.reshape(features_test.shape[0],-1)

print(flatten_train.shape)
print(flatten_test.shape)

(585, 13532)
(148, 13532)


#  Classifiers
***
*  **SVM**
*  **KNN**
*  **Forest Classifier**




SVM

In [12]:
from sklearn import svm

clf = svm.SVC(C= 50.0, kernel='rbf', gamma='scale',
                coef0=0.0, shrinking=True, probability=False,tol=0.001,
                cache_size=200, class_weight=None, verbose=False, max_iter=-1,
                decision_function_shape='ovr', break_ties=True, random_state=None)
clf.fit(flatten_train,labels_train)

C:\Users\Sahil Jindal\anaconda3\envs\FirstEnv\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVC(C=50.0, break_ties=True)

In [13]:
predicted_test = clf.predict(flatten_test)

In [14]:
predicted_test = predicted_test.reshape((len(predicted_test),1))
print(predicted_test.shape)

(148, 1)


In [15]:
import sklearn
acc_test = sklearn.metrics.accuracy_score(predicted_test,labels_test)
print(acc_test)

0.6756756756756757


KNN

In [16]:
from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(n_neighbors = 4, weights = 'distance') 
clf.fit(flatten_train,labels_train)

<ipython-input-16-a7ae32e8bd17>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  clf.fit(flatten_train,labels_train)


KNeighborsClassifier(n_neighbors=4, weights='distance')

In [17]:
predicted_test = clf.predict(flatten_test)
predicted_test = predicted_test.reshape((len(predicted_test),1))
print(predicted_test.shape)


import sklearn
acc_test = sklearn.metrics.accuracy_score(predicted_test,labels_test)
print(acc_test)

(148, 1)
0.3716216216216216


Random Forest

In [43]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None,
                             min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                             max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0,
                             min_impurity_split=None, bootstrap=True, oob_score=False,
                             n_jobs=None, random_state=None, verbose=0, warm_start=False,
                             class_weight=None, ccp_alpha=0.0, max_samples=None)

clf.fit(flatten_train,labels_train)

<ipython-input-43-122b5b4f467f>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(flatten_train,labels_train)


RandomForestClassifier()

In [44]:
predicted_test = clf.predict(flatten_test)
predicted_test = predicted_test.reshape((len(predicted_test),1))
print(predicted_test.shape)


import sklearn
acc_test = sklearn.metrics.accuracy_score(predicted_test,labels_test)
print(acc_test)

(148, 1)
0.7972972972972973


### Multi-Layer Perceptron Model

In [20]:
import keras
import tensorflow as tf
#from sklearn.preprocessing import LabelBinarizer

### Changing labels to one-hot encoded vector

In [21]:
num_classes = 5

In [22]:
y_train = keras.utils.to_categorical(labels_train, num_classes)
y_test = keras.utils.to_categorical(labels_test, num_classes)

In [23]:
print(y_train.shape)

(585, 5)


### features

In [24]:
X_train = flatten_train
X_test = flatten_test


In [25]:
print('train shape = ', X_train.shape)
print('test shape = ', X_test.shape)


train shape =  (585, 13532)
test shape =  (148, 13532)


In [26]:
n_features  = X_train.shape[1]
print("number of features = ", n_features)

number of features =  13532


### Model

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.optimizers import Adam,RMSprop

In [28]:
from tensorflow.python.client import device_lib



In [29]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8965768256378696759
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16495748590638932985
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 3429760771964709666
physical_device_desc: "device: XLA_GPU device"
]


In [30]:
model = Sequential()
model.add(Dense(8192, activation='relu', input_shape=(n_features,)))
model.add(Dropout(0.2))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8192)              110862336 
_________________________________________________________________
dropout (Dropout)            (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              33558528  
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               2097664   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 2

In [31]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [32]:
batch_size = 32
num_classes = 5
epochs = 25

In [37]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='E:\Baby Cry', mode='max', monitor='val_accuracy', verbose=2, save_best_only=True)
callbacks_list = [checkpoint]

In [38]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=callbacks_list,
                    verbose=1,
                    validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/25
19/19 [==============================] - ETA: 0s - loss: 204.2023 - accuracy: 0.5248
Epoch 00001: val_accuracy improved from -inf to 0.30405, saving model to E:\Baby Cry
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: E:\Baby Cry\assets
19/19 [==============================] - 67s 4s/step - loss: 204.2023 - accuracy: 0.5248 - val_loss: 177.0604 - val_accuracy: 0.3041
Epoch 2/25
19/19 [==============================] - ETA: 0s - loss: 138.5193 - accuracy: 0.6068
Epoch 00002: val_accuracy improved from 0.30405 to 0.47297, saving model to E:\Baby Cry
INFO:tensorflow:Assets written to: E:\Baby Cry\assets
19/19 [==============================] - 66s 3s/step - loss: 138.5193 - accuracy: 0.6068 - val_loss: 106.4988 - val_accuracy: 0.4730
Epoch 3/25
19/19 [==============================] - ETA: 0s - loss: 128.3923 - accuracy: 0.6137
Epoch 00003: val_accuracy did not improve from 0.47297
19/19 [======================